In [3]:
import ee
import pandas as pd
import requests
from io import StringIO

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# URL of the CSV file
url = 'https://raw.githubusercontent.com/orwell2024/uscrnlib/main/extract_slides/2024stations_days.csv'

# Read the CSV file from the URL
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# Limiting to first 10 locations for testing purposes
data = data.head(4)

# Function to create and export image for a given location
def create_and_export_image(row):
    station_name = row['Station']
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    
    point = ee.Geometry.Point([longitude, latitude])
    buffer = point.buffer(10000).bounds()  # 5000 meters buffer for 10 km x 10 km region
    
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterBounds(buffer) \
                    .filterDate('2023-04-11', '2023-6-01') \
                    .sort('CLOUDY_PIXEL_PERCENTAGE') \
                    .first()
    
    # Get the date of the image
    date = ee.Date(collection.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    
    # Select the relevant bands
    image = collection.select(['B4', 'B3', 'B2']).clip(buffer)
    
    # Export the image with smaller dimensions
    export_task = ee.batch.Export.image.toDrive(
        image=image,
        description=f"{station_name}_{date}",
        folder='GEE_Images',
        scale=10,
        region=buffer.getInfo()['coordinates'],
        fileFormat='GeoTIFF',  # Export as GeoTIFF
        maxPixels=1e8
    )
    
    export_task.start()

# Apply the function to each row in the dataframe
data.apply(create_and_export_image, axis=1)

print("Export tasks have been started.")


Export tasks have been started.


In [16]:
import ee
import pandas as pd
import requests
from io import StringIO

# Initialize the Earth Engine module.
# ee.Authenticate()
ee.Initialize()

# URL of the CSV file
url = 'https://raw.githubusercontent.com/orwell2024/uscrnlib/main/extract_slides/2024stations_days.csv'

# Read the CSV file from the URL
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# Filter the data to include only stations from Alabama (AL)
data_al = data[data['Station'].str.startswith('TX_')].copy()

# Initialize lists to store results
built_1975_percent_list = []
built_2020_percent_list = []
percentage_change_list = []

# Function to create and export image for a given location
def process_location(row):
    station_name = row['Station']
    latitude = row['LATITUDE']
    longitude = row['LONGITUDE']
    sizeKm = 10  # Size of the cell in kilometers

    # Define a point for the center of the rectangle at the specified coordinates
    centerPoint = ee.Geometry.Point([longitude, latitude])

    # Create a bounding box around the center point
    halfSideLength = (sizeKm / 2) * 1000  # Convert km to meters
    cell = centerPoint.buffer(halfSideLength).bounds()

    # Load the built-up surface images for 1975 and 2020 from the JRC GHSL dataset
    image_1975 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/1975').select('built_surface')
    image_2020 = ee.Image('JRC/GHSL/P2023A/GHS_BUILT_S/2020').select('built_surface')

    # Clip the built-up images to the cell
    built_1975_clipped = image_1975.clip(cell)
    built_2020_clipped = image_2020.clip(cell)

    # Calculate the average built-up value for the cell in 1975
    mean1975 = built_1975_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Calculate the average built-up value for the cell in 2020
    mean2020 = built_2020_clipped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=cell,
        scale=30,
        maxPixels=1e9
    ).get('built_surface').getInfo()

    # Normalize to percentage of the area (1% = 10,000 square meters per hectare)
    percentage1975 = round((mean1975 / 10000) * 100, 2) if mean1975 is not None else 0
    percentage2020 = round((mean2020 / 10000) * 100, 2) if mean2020 is not None else 0

    # Calculate the percentage change
    percentage_change = round(((percentage2020 - percentage1975) / percentage1975) * 100, 2) if percentage1975 != 0 else None

    # Append results to lists
    built_1975_percent_list.append(percentage1975)
    built_2020_percent_list.append(percentage2020)
    percentage_change_list.append(percentage_change)

    # Export the 1975 image to Google Drive
    export_task_1975 = ee.batch.Export.image.toDrive(
        image=built_1975_clipped,
        description=f'Built_up_surface_1975_{station_name}_{sizeKm}km_cell',
        folder='GEE_Images',
        scale=30,
        region=cell,
        maxPixels=1e9
    )
    export_task_1975.start()

    # Export the 2020 image to Google Drive
    export_task_2020 = ee.batch.Export.image.toDrive(
        image=built_2020_clipped,
        description=f'Built_up_surface_2020_{station_name}_{sizeKm}km_cell',
        folder='GEE_Images',
        scale=30,
        region=cell,
        maxPixels=1e9
    )
    export_task_2020.start()

# Apply the function to each row in the filtered dataframe
data_al.apply(process_location, axis=1)

# Add the results to the dataframe
data_al.loc[:, 'Built_1975_percent'] = built_1975_percent_list
data_al.loc[:, 'Built_2020_percent'] = built_2020_percent_list
data_al.loc[:, 'Percentage_Change'] = percentage_change_list

# Save the updated dataframe to a new CSV file
output_csv_path = 'updated_stations_data_al.csv'
data_al.to_csv(output_csv_path, index=False)

print("Export tasks have been started and results have been written to the CSV file.")


Export tasks have been started and results have been written to the CSV file.
